# Matrix inverses in Julia

## QR factorization

- the `qr` command finds the QR factorization of a matrix

In [1]:
A = rand(5, 3)
Q, R = qr(A);

In [2]:
Q

5×3 Array{Float64,2}:
 -0.61213   -0.194461    0.594399 
 -0.23158   -0.21042    -0.686179 
 -0.378558  -0.540393   -0.321692 
 -0.451696   0.79047    -0.259278 
 -0.473637   0.0322663   0.0716775

In [3]:
R

3×3 Array{Float64,2}:
 -1.62915  -1.20081   -1.18793   
  0.0       0.513539   0.00376739
  0.0       0.0       -0.956655  

- when columns of $n \times k$ matrix $A$ are independent, `qr` is same as ours
- when columns are dependent, `qr` is *not* same as ours
  - $A = QR, Q^T Q = I$ and $R_{ij} = 0$ for $i > j$ always hold
  - $R$ can have zero or negative diagonal entries
  - $R$ is not square when $A$ is wide

## checking linear independence with QR

Let's check if columns of $A$ are linearly independent

- $A$ must be tall or square
- columns are linearly independent if and only if $R$ has no 0 diagonal entries

Check if columns of (tall or square) $A$ are linearly independent

In [4]:
a1 = rand(5)

5-element Array{Float64,1}:
 0.376955 
 0.0556314
 0.450979 
 0.155615 
 0.206433 

In [5]:
a2 = rand(5)

5-element Array{Float64,1}:
 0.159075
 0.931782
 0.569187
 0.565357
 0.924822

In [6]:
A = [a1 a2 a1+a2]

5×3 Array{Float64,2}:
 0.376955   0.159075  0.53603 
 0.0556314  0.931782  0.987413
 0.450979   0.569187  1.02017 
 0.155615   0.565357  0.720972
 0.206433   0.924822  1.13126 

In [7]:
Q, R = qr(A);

In [8]:
Q

5×3 Array{Float64,2}:
 -0.584864    0.364207   0.221857 
 -0.0863148  -0.71847    0.662085 
 -0.699716    0.113619   0.0852882
 -0.241444   -0.27447   -0.085246 
 -0.320291   -0.512748  -0.705607 

In [9]:
R

3×3 Array{Float64,2}:
 -0.644518  -1.00445  -1.64897    
  0.0       -1.17623  -1.17623    
  0.0        0.0      -2.28983e-16

find the entry of diagonal of $R$ closest to zero ($R$ can have negative entries)

In [10]:
minimum(abs(diag(R)))

2.289834988289386e-16

In [11]:
indmin(abs(diag(R)))

3

## Inverse

In [12]:
inv(A)

LoadError: DimensionMismatch("matrix is not square")

In [13]:
inv(R)

3×3 Array{Float64,2}:
 -1.55155   1.32496    4.36713e15
  0.0      -0.850177   4.36713e15
  0.0       0.0       -4.36713e15

You can solve square set of linear equations $Ax = b$, with invertible $A$, using

In [14]:
b = rand(5,1)
A = rand(5,5)
x = inv(A) * b

5×1 Array{Float64,2}:
  0.998329
 -6.43711 
  7.46487 
 -2.85659 
  3.9032  

In [15]:
norm(A*x - b) # check residual

1.5700924586837751e-15

but there is a better way using backslash

## Pseudo-inverse

- for a $m \times n$ matrix $A$, `pinv(A)` returns the $n \times m$ pseudo-inverse
- if $A$ is square and invertible
  - `pinv(A)` returns the inverse $A^{-1}$
- if $A$ is tall with linearly independent columns
  - `pinv(A)` returns the left inverse $(A^T A)^{-1} A^T$
- if $A$ is wide with linearly independent rows
  - `pinv(A)` returns the right inverse $A^T (A A^T)^{-1}$
- in other cases, `pinv(A)` returns an $m \times n$ matrix, but
  - it is not a left or right inverse of $A$
  - what it is is beyond the scope of this class (ee103)

## The backslash operator

- given $A$ and $b$, the `\` operator solves the linear system $Ax=b$ for $x$
- for a $m \times n$ matrix $A$ and a $m$-vector $b$, `A \ b` returns a $n$-vector $x$
- if $A$ is square and invertible
  - $x=A^{-1} b$
  - the unique solution of $Ax=b$
- if $A$ is tall with linearly independent columns
  - $x = (A^T A)^{-1} A^T b$
  - the least squares approximate solution of $Ax=b$
- if $A$ is wide with linearrly independent rows
  - $x=A^T (A A^T)^{-1} b$
  - $x$ is the least norm solution of $Ax=b$
- in other cases, `A\b` will print an error message
- uses a factor and solve method similar to QR

## Solving matrix systems with backslash

- solve matrix equatin $AX=B$ for $X$, with $A$ square
- with $X = [x_1 \dots x_k]$, $B=[b_1 \dots b_k]$, same as solving k linear systems

$$
Ax_1=b_1, \dots, Ax_k=b_k
$$

- `X = A\B` solves the system, doing the right thing:
  - factor $A$ once (order $n^3$)
  - back substitution to get $x_i = A^{-1} b_i, i=1,\dots,k$ (order $kn^2$)